In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from osgeo import gdal
from sentinelhub import AwsProductRequest

In [2]:
import pandas as pd

In [3]:
from datetime import date

In [4]:
import requests

In [5]:
import threading
import multiprocessing

In [6]:
import os
import copy
import shutil
from pathlib import Path
from glob import glob
import zipfile

In [7]:
def get_credential(id_name):
    file_path = os.getcwd() + '/credentials.sh'
    f = open(file_path, 'r+')
    lines = f.readlines()
    for line in lines:
        ids = line.strip().split('=')
        if ids[0] == id_name:
            return ids[1].strip()
    return 'Not found'

In [8]:
USERNAME = [get_credential("username"), get_credential("username1")]
PASSWORD = [get_credential("password"), get_credential("password1")]
API_SOURCE = "https://scihub.copernicus.eu/dhus"
PROCESSING_LEVEL = ["Level-1", "Level-2"]
SATELLITE = "Sentinel-2"
STUDY_AREA = "GodavariNew.geojson"
REGION = "Godavari"
CWD = ""+os.getcwd()

In [53]:
FEB_2016 = "20160209"
FEB_END16 = "20160308"

FEB_2017 = "20170209"
FEB_END17 = "20170308"

FEB_2018 = "20180209"
FEB_END18 = "20180308"

FEB_2019 = "20190209"
FEB_END19 = "20190308"

FEB_2020 = "20200120"
FEB_END20 = "20200318"

In [54]:
months = [
    "Feb16", "Feb17", "Feb18", "Feb19", "Feb20"
]

In [55]:
#dates = [[FEB_2019, FEB_END19], [FEB_2020, FEB_END20]]
dates = [[FEB_2016, FEB_END16], [FEB_2017, FEB_END17], [FEB_2018, FEB_END18], [FEB_2019, FEB_END19], [FEB_2020, FEB_END20]]

In [56]:
footprint = geojson_to_wkt(read_geojson(STUDY_AREA))

In [57]:
api = []
for i in range(len(USERNAME)):
    api.append(SentinelAPI(USERNAME[i], PASSWORD[i], API_SOURCE))

In [58]:
def queryForProducts(footprint, beg, end, sat):
    return api[0].query(footprint, date=(beg, end),platformname=sat)

In [59]:
def getLevelURLs(df):
    level1 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[0] in df["processinglevel"][i]:
            level1.append(df["link"][i])

    level2 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[1] in df["processinglevel"][i]:
            level2.append(df["link"][i])
    
    return level1, level2

In [60]:
def curateLevelProducts(products, df, level):
    keys = list(products.keys())
    for i in range(df.shape[0]):
        if level not in df["processinglevel"][i]:
            products.pop(keys[i])
    return products

In [61]:
def createDataDirectory(name):
    data_dir = os.getcwd()+"\\data\\"+name+"_data"
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    return data_dir
def getDataDirectory(name):
    return os.getcwd()+"\\data\\"+name+"_data"
def createArchiveDirectory(name):
    data_dir = os.getcwd()+"\\data\\"+name+"_archive"
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    return data_dir
def getArchiveDirectory(name):
    return os.getcwd()+"\\data\\"+name+"_archive"
def createBackupDirectory(name):
    backup = "D:\\"+name+"_data"
    if not os.path.exists(backup):
        os.mkdir(backup)
    return backup
def getBackupDirectory(name):
    return "D:\\"+name+"_data"

In [62]:
def listDataFiles(data_dir, ext):
    paths = glob(data_dir+"/*"+ext)
    fileNames = []
    for path in paths:
        fileNames.append(path.split('.')[0].split('\\')[-1])
    return paths, fileNames

In [63]:
def getDesiredFiles(data_det):
    product_ids = (list(data_det.keys()))
    desired_files = []
    for pid in product_ids:
        desired_files.append(data_det[pid]['title'])
    return desired_files

In [64]:
def checkAllFiles(data_det):
    desired_files = getDesiredFiles(data_det)
    for file_name in desired_files:
        date = file_name.split('_')[2][:6]
        file_path = getDataDirectory(REGION+str(date))+"\\"+file_name+".SAFE"
        if not os.path.exists(file_path):
            print(file_path)
    print("All other files downloaded")

In [65]:
def extractFiles(data_dir):
    paths, fileNames = listDataFiles(data_dir, ".zip")
    l = len(paths)
    i = 0
    for path in paths:
        print(str(i)+"/"+str(l)+"  Extracting "+path.split('.')[0].split('\\')[-1]+"...")
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)

In [66]:
def getOnlineProducts(data_det):
    online_products = []
    for products in data_det:
        for prod in products:
            if api[0].get_product_odata(prod)['Online']:
                online_products.append(products[prod]['title'])
    return online_products

In [67]:
def cleanup(data_dir):
    paths, fileNames = listDataFiles(data_dir, ".zip")
    l = len(paths)
    i = 0
    for path in paths:
        print(str(i)+"/"+str(l)+"Deleting "+path.split('.')[0].split('\\')[-1]+"...")
        os.remove(path)

In [68]:
def getFolders(data_dir):
    #return all  data folders  list
    return
def manipualateFolders(data_dir):
    #extract  annd cleannn for  all get folders
    return

In [69]:
def getData(products):
    CWD = Path(os.getcwd())
    try:
        date = products[list(products.keys())[0]]['title'].split('_')[2][:4]
        path = createDataDirectory(REGION+str(date))
        if not api[0].get_product_odata(prod)['Online']:
            path = createArchiveDirectory(REGION+str(date))
        print("Changing Directory " + path)
        os.chdir(path)
        for prod in products:
            if api[0].get_product_odata(prod)['Online'] and (not os.path.exists(path+'\\'+products[prod]['title']+'.zip')):
                print(prod+"----"+products[prod]['title'])
                api[0].download(prod)
            elif os.path.exists(path+'\\'+products[prod]['title']+'.zip'):
                print(products[prod]['title'])
            else:
                #print(products[prod]['title'])
                print(prod+" not online. Downloading using AWS EC2 instance :)")
                #product_request = AwsProductRequest(product_id=products[prod]['title'],
                                    #data_folder=path, safe_format=True)
                #product_request.get_data(save_data=True)                
        #path=path+file_name
        #backupPath = createBackupDirectory(REGION+str(date))
        #print("Backup " + file_name + " to " + backupPath)
        #shutil.copy2(path,backupPath)
        #extractFile(path)
        #cleanup(path)
        os.chdir(CWD)
    except AssertionError:
        os.chdir(CWD)
        print(os.getcwd())
        print("Download of specified products done")
    except Exception as e:
        os.chdir(CWD)
        print(os.getcwd())
        print("Something went wrong " + str(e))

In [70]:
products = []
for date in dates:
    products.append(queryForProducts(footprint, date[0], date[1],SATELLITE))

In [71]:
products_df  =  []
level1 = []
level2 = []
for product in products:
    products_df.append(api[0].to_dataframe(product))
    l1, l2 = getLevelURLs(products_df[-1])
    level1.append(l1)
    level2.append(l2)

In [72]:
data_det = []
for i in range(len(products_df)):
    temp = copy.deepcopy(products[i])
    det_l2 = curateLevelProducts(products[i], products_df[i], PROCESSING_LEVEL[1])
    det_l1 = curateLevelProducts(temp, products_df[i], PROCESSING_LEVEL[0])
    if len(det_l2) >= len(det_l1):
        data_det.append(det_l2)
    else:
        data_det.append(det_l1)
    print(len(data_det[-1]))

2
6
18
19
38


In [73]:
levelCount = []

for i in range(len(products_df)):
    counts = [products_df[i].shape[0], len(level1[i]), len(level2[i])]
    levelCount.append(counts)

print("      Total  L1  L2")
for i in range(len(levelCount)):
    print(months[i] + "   " + str(levelCount[i][0]) + "     " + str(levelCount[i][1]) + "  " + str(levelCount[i][2]))

      Total  L1  L2
Feb16   2     2  0
Feb17   6     6  0
Feb18   18     18  0
Feb19   38     19  19
Feb20   76     38  38


In [74]:
data_dir = []
for i in range(len(data_det)):
    getData(data_det[i])
    #checkAllFiles(data_det[i])

Changing Directory C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2016_data
S2A_MSIL1C_20160213T044912_N0201_R076_T44QPD_20160213T050114
S2A_MSIL1C_20160213T044912_N0201_R076_T44QPE_20160213T050114
Changing Directory C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2017_data
S2A_MSIL1C_20170302T045711_N0204_R119_T44QPE_20170302T050237
S2A_MSIL1C_20170302T045711_N0204_R119_T44QPD_20170302T050237
S2A_MSIL1C_20170302T045711_N0204_R119_T44QND_20170302T050237
S2A_MSIL1C_20170210T045931_N0204_R119_T44QND_20170210T050445
S2A_MSIL1C_20170210T045931_N0204_R119_T44QPE_20170210T050445
S2A_MSIL1C_20170210T045931_N0204_R119_T44QPD_20170210T050445
Changing Directory C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\Godavari2018_data
S2A_MSIL1C_20180307T045651_N0206_R119_T44QND_20180307T075359
S2A_MSIL1C_20180307T045651_N0206_R119_T44QPE_20180307T075359
S2A_MSIL1C_20180307T045651_N0206_R119_T44QPD_20180307T075359
S2B_MSIL1C_20180302T045709_N0206